<a href="https://colab.research.google.com/github/KhushiBhambri/YogNet/blob/55_Khushi/PreProcessingVideos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Major_Project_Cse"

Mounted at /content/drive
/content/drive/MyDrive/Major_Project_Cse


In [ ]:
import os,cv2
import numpy as np
import pandas as pd
# import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

data_path = './Videos'

##POSE STREAM

In [ ]:
!pip install mediapipe

import mediapipe as mp
from google.colab.patches import cv2_imshow

mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8,model_complexity=2)

# Setup the Pose function for videos - for video processing.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 17.6 MB/s eta 0:00:00


In [ ]:
def mpPoseImg(image_pose, pose=pose_image, draw=False, display=False):
    
    annotated_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
 
    
    results = pose.process(image_in_RGB)

    if results.pose_landmarks and draw:    
      mp_drawing.draw_landmarks(image=annotated_image,landmark_list=results.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


        # mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
        #                           landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),thickness=3, circle_radius=3),
        #                           connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),thickness=2, circle_radius=2))

    if display:
            
            # plt.figure(figsize=[22,22])
            # plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            # plt.subplot(122);plt.imshow(annotated_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');
            cv2_imshow(annotated_image)
            return results

    else:
        
        return annotated_image,results

In [ ]:
numOfKeypoints = 34
def getKeyPointsforFrame(frame):
  img,results = mpPoseImg(frame)
  keypoints = [[None,None] for _ in range(numOfKeypoints)]
  if results.pose_landmarks == None:
    return keypoints
    
  landmarks = results.pose_landmarks
  for i in range(numOfKeypoints-2):
     mark = landmarks.landmark[i]
     keypoints[i][0]=mark.x
     keypoints[i][1]=mark.y
  #------------------------------------------------------------------------------------------------
  try:
    # MID HIP COORDINATES
      left_hip_index = 23
      right_hip_index = 24
      Mid_hip_x =  (keypoints[left_hip_index][0]+keypoints[right_hip_index][0])/2
      Mid_hip_y =(keypoints[left_hip_index][1]+keypoints[right_hip_index][1])/2
      # keypoints.append([Mid_hip_x,Mid_hip_y])
      keypoints[32][0]=Mid_hip_x
      keypoints[32][1]=Mid_hip_y
                          
  except:
      pass
      
  #------------------------------------------------------------------------------------------------
  try:
    # NECK COORDINATES
      left_shoulder_index = 11
      right_shoulder_index = 12
      Neck_x =  (keypoints[left_shoulder_index][0]+keypoints[right_shoulder_index][0])/2
      Neck_y =(keypoints[left_shoulder_index][1]+keypoints[right_shoulder_index][1])/2
      # keypoints.append([Neck_x,Neck_y])
      keypoints[33][0]=Neck_x
      keypoints[33][1]=Neck_y
                          
  except:
      pass

  #------------------------------------------------------------------------------------------------
     
  return keypoints

In [ ]:
imagePath='Images/yoga.jpg'
img = cv2.imread(imagePath)
getKeyPointsforFrame(img)

In [ ]:

labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107


train_videos=[]
train_labels=[]
train_keypoints = []
import traceback
import random

num = 1

for i,folder in enumerate(labels):
  try:
    for video in os.listdir(data_path+'/'+folder):
      print(num)
      num+=1
      video = os.path.join(data_path+'/'+folder+'/'+video)
      # Open the video file
      cap = cv2.VideoCapture(video)

      # Get the total number of frames in the video
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

      # Select a random starting frame number
      start_frame = random.randint(0, total_frames - 16)

      # Set the number of frames to be selected
      num_frames = 16

      # Set the frame number to start with
      cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

      # Loop through the frames and save them
      frames = []
      keypoints = []
      for j in range(num_frames):
          ret, image = cap.read()
          if ret:
            frames.append(cv2.resize(image,(112,112)))
            kk = getKeyPointsforFrame(image)
            keypoints.append(kk)

      # Release the video file
      cap.release()


      # train_videos.append(frames)
      train_keypoints.append(keypoints)
      train_labels.append(i)
      # if(num>2):
      #    break

  except Exception:
    traceback.print_exc()


In [ ]:

# create a DataFrame to store the data
data = pd.DataFrame({'keypoints': train_keypoints, 'label': train_labels})

# save the DataFrame to a CSV file
data.to_csv('train_data.csv', index=False)

# RGB STREAM


In [ ]:

labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 20

In [ ]:

train_videos=[]
train_labels=[]
import traceback
import random

num = 1

for i,folder in enumerate(labels):
    try:
      for video in os.listdir(data_path+'/'+folder):
        print(num)
        num+=1
        video = os.path.join(data_path+'/'+folder+'/'+video)
        # Open the video file
        cap = cv2.VideoCapture(video)

        # Get the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Select a random starting frame number
        start_frame = random.randint(0, total_frames - 16)

        # Set the number of frames to be selected
        num_frames = 16

        # Set the frame number to start with
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # Loop through the frames and save them
        frames = []
        for j in range(num_frames):
            ret, image = cap.read()
            if ret:
              frames.append(cv2.resize(image,(112,112)))

        # Release the video file
        cap.release()

        train_videos.append(frames)
        train_labels.append(i)

    except Exception:
      traceback.print_exc()
train_videos = np.asarray(train_videos)
train_labels = np.asarray(train_labels).astype('int64')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
# create a DataFrame to store the data
data = pd.DataFrame({'video': train_videos, 'label': train_labels})

# save the DataFrame to a CSV file
data.to_csv('Rgb_train_data.csv', index=False)

In [12]:
[type(train_videos[0][0][0][0])]

[numpy.ndarray]

In [14]:
train_videos[0]

[array([[[151, 133, 137],
         [183, 171, 173],
         [192, 192, 192],
         ...,
         [188, 181, 173],
         [178, 171, 163],
         [178, 173, 165]],
 
        [[150, 133, 136],
         [184, 172, 173],
         [189, 189, 189],
         ...,
         [187, 181, 176],
         [178, 172, 167],
         [131, 123, 118]],
 
        [[145, 128, 132],
         [180, 169, 171],
         [183, 184, 185],
         ...,
         [192, 185, 179],
         [126, 117, 111],
         [127, 115, 110]],
 
        ...,
 
        [[ 39,  27,  33],
         [ 54,  39,  42],
         [ 81,  69,  74],
         ...,
         [189, 171, 169],
         [212, 189, 182],
         [221, 191, 185]],
 
        [[ 36,  26,  33],
         [ 48,  37,  41],
         [ 82,  69,  73],
         ...,
         [176, 157, 156],
         [216, 193, 186],
         [221, 192, 181]],
 
        [[ 36,  26,  33],
         [ 48,  36,  41],
         [ 83,  68,  69],
         ...,
         [170, 150, 152],
  